In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 7.5 MB/s eta 0:00:00


In [3]:
!pip install catboost


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 5.6 MB/s eta 0:00:00


In [4]:
!pip install scikeras

In [5]:
# Import libraries
import optuna
import pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier

# Load datasets
main_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/9_PCP (Physicochemical Properties)/positive_main_pcp.csv")
main_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/9_PCP (Physicochemical Properties)/negative_main_pcp.csv")
validation_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/9_PCP (Physicochemical Properties)/positive_validation_pcp.csv")
validation_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/9_PCP (Physicochemical Properties)/negative_validation_pcp.csv")

# Prepare features and labels for training
X_train = pd.concat([main_p, main_n])
y_train = pd.concat([pd.Series(1, index=main_p.index), pd.Series(0, index=main_n.index)])


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [6]:
# Define a cross-validation accuracy function
def cross_val_accuracy(model, X_train, y_train):
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    accuracies = cross_val_score(model, X_train, y_train, cv=cv, scoring='accuracy')
    return accuracies.mean()


In [7]:
#Support Vector Machine (SVM)
def objective_svm(trial):
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly'])
    C = trial.suggest_loguniform('C', 1e-5, 1e5)
    gamma = trial.suggest_loguniform('gamma', 1e-5, 1e-1)

    model = SVC(kernel=kernel, C=C, gamma=gamma)
    return cross_val_accuracy(model, X_train, y_train)

study_svm = optuna.create_study(direction='maximize')
study_svm.optimize(objective_svm, n_trials=50)
print("SVM Best Parameters:", study_svm.best_params)
print("SVM Best Accuracy:", study_svm.best_value)


[I 2024-12-07 09:57:27,252] A new study created in memory with name: no-name-50ba81b2-38f3-43a2-8ca8-b6f807b78520
<ipython-input-7-cd4fae0f87b1>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-5, 1e5)
<ipython-input-7-cd4fae0f87b1>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  gamma = trial.suggest_loguniform('gamma', 1e-5, 1e-1)
[I 2024-12-07 09:57:27,443] Trial 0 finished with value: 0.5250073997336095 and parameters: {'kernel': 'poly', 'C': 676.5883933711389, 'gamma': 1.8659436436985857e-05}. Best is trial 0 with value: 0.5250073997336095.
<ipython-input-7-cd4fae0f87b1>:4: FutureWarning: suggest_loguniform has been deprecated 

SVM Best Parameters: {'kernel': 'rbf', 'C': 26682.22058742274, 'gamma': 0.09588397871449833}
SVM Best Accuracy: 0.8049726209856445


In [8]:
#Random Forest
def objective_rf(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    max_depth = trial.suggest_int('max_depth', 3, 30)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf
    )
    return cross_val_accuracy(model, X_train, y_train)

study_rf = optuna.create_study(direction='maximize')
study_rf.optimize(objective_rf, n_trials=50)
print("Random Forest Best Parameters:", study_rf.best_params)
print("Random Forest Best Accuracy:", study_rf.best_value)


[I 2024-12-07 10:06:28,448] A new study created in memory with name: no-name-97f98211-2fd8-4f64-bc8c-db8f39679079
[I 2024-12-07 10:06:30,682] Trial 0 finished with value: 0.8273013171525825 and parameters: {'n_estimators': 162, 'max_depth': 7, 'min_samples_split': 9, 'min_samples_leaf': 2}. Best is trial 0 with value: 0.8273013171525825.
[I 2024-12-07 10:06:32,755] Trial 1 finished with value: 0.8445019979280748 and parameters: {'n_estimators': 153, 'max_depth': 14, 'min_samples_split': 10, 'min_samples_leaf': 3}. Best is trial 1 with value: 0.8445019979280748.
[I 2024-12-07 10:06:35,965] Trial 2 finished with value: 0.8109923042770462 and parameters: {'n_estimators': 261, 'max_depth': 13, 'min_samples_split': 7, 'min_samples_leaf': 10}. Best is trial 1 with value: 0.8445019979280748.
[I 2024-12-07 10:06:37,815] Trial 3 finished with value: 0.8419268906319373 and parameters: {'n_estimators': 137, 'max_depth': 18, 'min_samples_split': 14, 'min_samples_leaf': 2}. Best is trial 1 with val

Random Forest Best Parameters: {'n_estimators': 227, 'max_depth': 30, 'min_samples_split': 3, 'min_samples_leaf': 1}
Random Forest Best Accuracy: 0.8642740861328992


In [9]:
#Decision Tree
def objective_dt(trial):
    max_depth = trial.suggest_int('max_depth', 3, 30)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

    model = DecisionTreeClassifier(
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf
    )
    return cross_val_accuracy(model, X_train, y_train)

study_dt = optuna.create_study(direction='maximize')
study_dt.optimize(objective_dt, n_trials=50)
print("Decision Tree Best Parameters:", study_dt.best_params)
print("Decision Tree Best Accuracy:", study_dt.best_value)


[I 2024-12-07 10:09:00,058] A new study created in memory with name: no-name-88c75cc3-fae4-4336-87ad-8647cc0a6a59
[I 2024-12-07 10:09:00,112] Trial 0 finished with value: 0.784368062749741 and parameters: {'max_depth': 16, 'min_samples_split': 10, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.784368062749741.
[I 2024-12-07 10:09:00,161] Trial 1 finished with value: 0.7792030486902471 and parameters: {'max_depth': 30, 'min_samples_split': 9, 'min_samples_leaf': 8}. Best is trial 0 with value: 0.784368062749741.
[I 2024-12-07 10:09:00,212] Trial 2 finished with value: 0.7852079325144294 and parameters: {'max_depth': 10, 'min_samples_split': 5, 'min_samples_leaf': 5}. Best is trial 2 with value: 0.7852079325144294.
[I 2024-12-07 10:09:00,262] Trial 3 finished with value: 0.7783261802575107 and parameters: {'max_depth': 12, 'min_samples_split': 17, 'min_samples_leaf': 5}. Best is trial 2 with value: 0.7852079325144294.
[I 2024-12-07 10:09:00,321] Trial 4 finished with value: 0.7800

Decision Tree Best Parameters: {'max_depth': 22, 'min_samples_split': 2, 'min_samples_leaf': 1}
Decision Tree Best Accuracy: 0.807566227615806


In [10]:
#k-Nearest Neighbors (k-NN)
def objective_knn(trial):
    n_neighbors = trial.suggest_int('n_neighbors', 3, 15)
    weights = trial.suggest_categorical('weights', ['uniform', 'distance'])
    algorithm = trial.suggest_categorical('algorithm', ['auto', 'ball_tree', 'kd_tree', 'brute'])

    model = KNeighborsClassifier(
        n_neighbors=n_neighbors,
        weights=weights,
        algorithm=algorithm
    )
    return cross_val_accuracy(model, X_train, y_train)

study_knn = optuna.create_study(direction='maximize')
study_knn.optimize(objective_knn, n_trials=50)
print("k-NN Best Parameters:", study_knn.best_params)
print("k-NN Best Accuracy:", study_knn.best_value)


[I 2024-12-07 10:09:22,984] A new study created in memory with name: no-name-b3705a48-2f5b-40fa-a052-1f893718d1a9
[I 2024-12-07 10:09:23,025] Trial 0 finished with value: 0.8152841497706083 and parameters: {'n_neighbors': 4, 'weights': 'distance', 'algorithm': 'kd_tree'}. Best is trial 0 with value: 0.8152841497706083.
[I 2024-12-07 10:09:23,123] Trial 1 finished with value: 0.7792104484238566 and parameters: {'n_neighbors': 9, 'weights': 'uniform', 'algorithm': 'ball_tree'}. Best is trial 0 with value: 0.8152841497706083.
[I 2024-12-07 10:09:23,218] Trial 2 finished with value: 0.7800762172561788 and parameters: {'n_neighbors': 15, 'weights': 'uniform', 'algorithm': 'ball_tree'}. Best is trial 0 with value: 0.8152841497706083.
[I 2024-12-07 10:09:23,258] Trial 3 finished with value: 0.8204491638301021 and parameters: {'n_neighbors': 14, 'weights': 'distance', 'algorithm': 'kd_tree'}. Best is trial 3 with value: 0.8204491638301021.
[I 2024-12-07 10:09:23,353] Trial 4 finished with valu

k-NN Best Parameters: {'n_neighbors': 15, 'weights': 'distance', 'algorithm': 'auto'}
k-NN Best Accuracy: 0.8221696018943317


In [11]:
#Logistic Regression

def objective_lr(trial):
    C = trial.suggest_loguniform('C', 1e-5, 1e5)
    penalty = trial.suggest_categorical('penalty', ['l2'])
    solver = trial.suggest_categorical('solver', ['lbfgs', 'saga'])

    model = LogisticRegression(C=C, penalty=penalty, solver=solver)
    return cross_val_accuracy(model, X_train, y_train)

study_lr = optuna.create_study(direction='maximize')
study_lr.optimize(objective_lr, n_trials=50)
print("Logistic Regression Best Parameters:", study_lr.best_params)
print("Logistic Regression Best Accuracy:", study_lr.best_value)


[I 2024-12-07 10:09:37,214] A new study created in memory with name: no-name-b1995bad-bf71-4506-8350-bf6ac2833fd3
<ipython-input-11-5e81b06dce9f>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-5, 1e5)
[I 2024-12-07 10:09:37,268] Trial 0 finished with value: 0.6761099600414384 and parameters: {'C': 1.9838889591396602e-05, 'penalty': 'l2', 'solver': 'saga'}. Best is trial 0 with value: 0.6761099600414384.
<ipython-input-11-5e81b06dce9f>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-5, 1e5)
[I 2024-12-07 10:09:37,326] Trial 1 finished with value: 0.6846825514281486 and parameters: {'C': 0.118954

Logistic Regression Best Parameters: {'C': 1.0275689135616877, 'penalty': 'l2', 'solver': 'saga'}
Logistic Regression Best Accuracy: 0.7052945093976616


In [12]:
#Naive Bayes
def objective_nb(trial):
    var_smoothing = trial.suggest_loguniform('var_smoothing', 1e-9, 1e-1)

    model = GaussianNB(var_smoothing=var_smoothing)
    return cross_val_accuracy(model, X_train, y_train)

study_nb = optuna.create_study(direction='maximize')
study_nb.optimize(objective_nb, n_trials=50)
print("Naive Bayes Best Parameters:", study_nb.best_params)
print("Naive Bayes Best Accuracy:", study_nb.best_value)


[I 2024-12-07 10:09:48,321] A new study created in memory with name: no-name-f40241a4-b331-452e-abdc-a20d8db8dd93
<ipython-input-12-5c7e648b4d2b>:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  var_smoothing = trial.suggest_loguniform('var_smoothing', 1e-9, 1e-1)
[I 2024-12-07 10:09:48,363] Trial 0 finished with value: 0.6683809382862217 and parameters: {'var_smoothing': 4.637883999227026e-06}. Best is trial 0 with value: 0.6683809382862217.
<ipython-input-12-5c7e648b4d2b>:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  var_smoothing = trial.suggest_loguniform('var_smoothing', 1e-9, 1e-1)
[I 2024-12-07 10:09:48,396] Trial 1 finished with value: 0.6683809382862217 and 

Naive Bayes Best Parameters: {'var_smoothing': 0.0616594909383373}
Naive Bayes Best Accuracy: 0.6838463815302649


In [13]:
#Gradient Boosting
def objective_gb(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    max_depth = trial.suggest_int('max_depth', 3, 20)
    subsample = trial.suggest_uniform('subsample', 0.5, 1.0)

    model = GradientBoostingClassifier(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        subsample=subsample
    )
    return cross_val_accuracy(model, X_train, y_train)

study_gb = optuna.create_study(direction='maximize')
study_gb.optimize(objective_gb, n_trials=50)
print("Gradient Boosting Best Parameters:", study_gb.best_params)
print("Gradient Boosting Best Accuracy:", study_gb.best_value)


[I 2024-12-07 10:09:55,978] A new study created in memory with name: no-name-a30633c3-4360-4874-82f9-b211bd3dfe74
<ipython-input-13-0f44ec352f28>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
<ipython-input-13-0f44ec352f28>:6: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  subsample = trial.suggest_uniform('subsample', 0.5, 1.0)
[I 2024-12-07 10:10:14,699] Trial 0 finished with value: 0.8539588574811307 and parameters: {'n_estimators': 146, 'learning_rate': 0.005132966961005172, 'max_depth': 16, 'subsample': 0.5186152344984405}. Best is trial 0 with value: 0.8539588574811307.
<ipython-input-13-0f44ec352f28>:4: Future

Gradient Boosting Best Parameters: {'n_estimators': 187, 'learning_rate': 0.03078396783840791, 'max_depth': 20, 'subsample': 0.5045820878431897}
Gradient Boosting Best Accuracy: 0.8642703862660944


In [14]:
#XGBoost

def objective_xgb(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    subsample = trial.suggest_uniform('subsample', 0.5, 1.0)

    model = XGBClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        subsample=subsample
    )
    return cross_val_accuracy(model, X_train, y_train)

study_xgb = optuna.create_study(direction='maximize')
study_xgb.optimize(objective_xgb, n_trials=50)
print("XGBoost Best Parameters:", study_xgb.best_params)
print("XGBoost Best Accuracy:", study_xgb.best_value)


[I 2024-12-07 10:22:17,425] A new study created in memory with name: no-name-a1b72741-d7e3-4c77-a408-05e971883561
<ipython-input-14-ad894fc3f237>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
<ipython-input-14-ad894fc3f237>:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  subsample = trial.suggest_uniform('subsample', 0.5, 1.0)
[I 2024-12-07 10:22:18,738] Trial 0 finished with value: 0.8359072073405358 and parameters: {'n_estimators': 123, 'max_depth': 14, 'learning_rate': 0.002006395577130386, 'subsample': 0.8493534581840485}. Best is trial 0 with value: 0.8359072073405358.
<ipython-input-14-ad894fc3f237>:6: Future

XGBoost Best Parameters: {'n_estimators': 168, 'max_depth': 12, 'learning_rate': 0.06323073039988844, 'subsample': 0.5521730118531978}
XGBoost Best Accuracy: 0.8668528932958413


In [15]:
# LightGBM
def objective_lgb(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    num_leaves = trial.suggest_int('num_leaves', 2, 50)

    model = LGBMClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        num_leaves=num_leaves
    )
    return cross_val_accuracy(model, X_train, y_train)

study_lgb = optuna.create_study(direction='maximize')
study_lgb.optimize(objective_lgb, n_trials=50)
print("LightGBM Best Parameters:", study_lgb.best_params)
print("LightGBM Best Accuracy:", study_lgb.best_value)


[I 2024-12-07 10:23:36,988] A new study created in memory with name: no-name-1ff1ef30-2906-4a6f-b4c2-33dc79108bb3
<ipython-input-15-f97698e6c3a7>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)


[LightGBM] [Info] Number of positive: 465, number of negative: 466
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000421 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1529
[LightGBM] [Info] Number of data points in the train set: 931, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499463 -> initscore=-0.002148
[LightGBM] [Info] Start training from score -0.002148
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

[I 2024-12-07 10:23:37,417] Trial 0 finished with value: 0.7637302057125944 and parameters: {'n_estimators': 72, 'max_depth': 5, 'learning_rate': 0.0004377647384418018, 'num_leaves': 42}. Best is trial 0 with value: 0.7637302057125944.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

<ipython-input-15-f97698e6c3a7>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
[I 2024-12-07 10:23:37,613] Trial 1 finished with value: 0.7285296729317745 and parameters: {'n_estimators': 80, 'max_depth': 15, 'learning_rate': 2.9229914942311903e-05, 'num_leaves': 11}. Best is trial 0 with value: 0.7637302057125944.
<ipython-input-15-f97698e6c3a7>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)


...

[LightGBM] [Info] Number of positive: 466, number of negative: 466
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000119 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1528
[LightGBM] [Info] Number of data points in the train set: 932, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Number of positive: 465, number of negative: 466
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000126 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1529
[LightGBM] [Info] Number of data points in the train set: 931, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499463 -> initscore=-0.002148
[LightGBM] [Info] Start training from score -0.002148
[Light

<ipython-input-15-f97698e6c3a7>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2024-12-07 10:24:12,078] Trial 49 finished with value: 0.8255919786887672 and parameters: {'n_estimators': 156, 'max_depth': 20, 'learning_rate': 0.011657172614372024, 'num_leaves': 44}. Best is trial 35 with value: 0.8668454935622318.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [16]:
#AdaBoost
def objective_ab(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)

    model = AdaBoostClassifier(
        n_estimators=n_estimators,
        learning_rate=learning_rate
    )
    return cross_val_accuracy(model, X_train, y_train)

study_ab = optuna.create_study(direction='maximize')
study_ab.optimize(objective_ab, n_trials=50)
print("AdaBoost Best Parameters:", study_ab.best_params)
print("AdaBoost Best Accuracy:", study_ab.best_value)


[I 2024-12-07 10:24:12,092] A new study created in memory with name: no-name-79a1df7f-d008-4e12-b4f2-81fc63c98a11
<ipython-input-16-95971a9e6870>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: Futu

AdaBoost Best Parameters: {'n_estimators': 185, 'learning_rate': 0.09373668644923759}
AdaBoost Best Accuracy: 0.7594309604854225


In [17]:
#Neural Network

import optuna
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler

# Load your dataset
# Replace this with your actual data loading process
X = np.random.rand(1000, 20)  # Dummy features
y = np.random.randint(0, 2, 1000)  # Dummy binary labels

# Normalize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

def objective_nn(trial):
    # Define hyperparameters
    num_layers = trial.suggest_int('num_layers', 1, 3)
    hidden_units = trial.suggest_int('hidden_units', 32, 512, step=32)
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
    epochs = trial.suggest_int('epochs', 10, 50)

    # Build the neural network
    model = Sequential()
    model.add(Dense(hidden_units, activation='relu', input_shape=(X.shape[1],)))
    model.add(Dropout(dropout_rate))

    for _ in range(num_layers - 1):
        model.add(Dense(hidden_units, activation='relu'))
        model.add(Dropout(dropout_rate))

    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # Train the model (without validation data)
    history = model.fit(X, y,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=0)

    # Evaluate the model
    train_accuracy = max(history.history['accuracy'])
    return train_accuracy

# Run Optuna study
study_nn = optuna.create_study(direction='maximize')
study_nn.optimize(objective_nn, n_trials=50)

# Print the best parameters and accuracy
print("Neural Network Best Parameters:", study_nn.best_params)
print("Neural Network Best Accuracy:", study_nn.best_value)

[I 2024-12-07 10:26:20,885] A new study created in memory with name: no-name-94f0ef41-e309-471f-9e54-48c5f1e21416
<ipython-input-17-565ccca4d2fe>:23: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
<ipython-input-17-565ccca4d2fe>:24: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activit

Neural Network Best Parameters: {'num_layers': 2, 'hidden_units': 288, 'dropout_rate': 0.13492301230033288, 'learning_rate': 0.004055445577915607, 'batch_size': 32, 'epochs': 48}
Neural Network Best Accuracy: 0.9959999918937683


In [18]:
#Multi-Layer Perceptron (MLP)

import optuna
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification

# Load your dataset (replace this with your actual dataset)
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2, random_state=42)

# Normalize the dataset
scaler = StandardScaler()
X = scaler.fit_transform(X)

def objective_mlp(trial):
    # Define the hyperparameters
    num_layers = trial.suggest_int('num_layers', 1, 3)
    hidden_units = trial.suggest_int('hidden_units', 32, 512, step=32)
    activation = trial.suggest_categorical('activation', ['relu', 'tanh', 'logistic'])
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
    epochs = trial.suggest_int('epochs', 50, 200)

    # Define the MLP model
    model = MLPClassifier(
        hidden_layer_sizes=(hidden_units,) * num_layers,
        activation=activation,
        learning_rate_init=learning_rate,
        max_iter=epochs,
        batch_size=batch_size,
        solver='adam',
        random_state=42
    )

    # Evaluate using cross-validation
    scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
    return scores.mean()

# Run Optuna study
study_mlp = optuna.create_study(direction='maximize')
study_mlp.optimize(objective_mlp, n_trials=50)

# Print results
print("MLP Best Parameters:", study_mlp.best_params)
print("MLP Best Accuracy:", study_mlp.best_value)

[I 2024-12-07 10:32:31,293] A new study created in memory with name: no-name-989f62f0-cdad-4e71-8507-9fcd494f04f2
<ipython-input-18-5ee0db595401>:21: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (198) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (198) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: S

MLP Best Parameters: {'num_layers': 3, 'hidden_units': 320, 'activation': 'tanh', 'learning_rate': 5.8323705599927455e-05, 'batch_size': 64, 'epochs': 105}
MLP Best Accuracy: 0.875


probabilites dataset

In [19]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
#from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Combine data and create labels
X_train = pd.concat([main_p, main_n], axis=0)
y_train = np.array([1] * len(main_p) + [0] * len(main_n))

X_validation = pd.concat([validation_p, validation_n], axis=0)
y_validation = np.array([1] * len(validation_p) + [0] * len(validation_n))

# Define models
models = {
    "RandomForest": RandomForestClassifier(random_state=42),
    "GradientBoosting": GradientBoostingClassifier(random_state=42),
    "LogisticRegression": LogisticRegression(max_iter=1000, random_state=42),
    "DecisionTree": DecisionTreeClassifier(random_state=42),
    "NaiveBayes": GaussianNB(),
    "k-NN": KNeighborsClassifier(),
    "SVM": SVC(probability=True, random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
    "LightGBM": LGBMClassifier(random_state=42),
    #"CatBoost": CatBoostClassifier(verbose=0, random_state=42),
    "AdaBoost": AdaBoostClassifier(random_state=42),
    "MLP": MLPClassifier(max_iter=500, random_state=42)
}

# Combined probabilities dataset
combined_probabilities = pd.DataFrame(index=X_validation.index)

# Train models and get probabilities
for model_name, model in models.items():
    print(f"Training {model_name}...")
    model.fit(X_train, y_train)
    probabilities = model.predict_proba(X_validation)[:, 1]
    combined_probabilities[model_name] = probabilities

# Neural Network
def build_nn(input_dim):
    model = Sequential([
        Dense(128, activation='relu', input_dim=input_dim),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Train Neural Network
input_dim = X_train.shape[1]
nn_model = build_nn(input_dim)
nn_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_validation, y_validation), verbose=1)

# Predict probabilities for Neural Network
nn_probabilities = nn_model.predict(X_validation).flatten()
combined_probabilities['NeuralNetwork'] = nn_probabilities

# Add ground truth
combined_probabilities['True_Label'] = y_validation

# Save the combined probabilities dataset
output_path = "PCP_optuna.csv"
combined_probabilities.to_csv(output_path, index=False)
print(f"Combined probabilities dataset with Neural Network saved to {output_path}")


Training RandomForest...
Training GradientBoosting...
Training LogisticRegression...
Training DecisionTree...
Training NaiveBayes...
Training k-NN...
Training SVM...
Training XGBoost...
Training LightGBM...
[LightGBM] [Info] Number of positive: 582, number of negative: 582
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000148 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1529
[LightGBM] [Info] Number of data points in the train set: 1164, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Training AdaBoost...


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [11:34:22] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Training MLP...
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.5382 - loss: 0.6783 - val_accuracy: 0.7133 - val_loss: 0.5787
Epoch 2/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6578 - loss: 0.5993 - val_accuracy: 0.7133 - val_loss: 0.5758
Epoch 3/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6829 - loss: 0.6018 - val_accuracy: 0.7200 - val_loss: 0.5760
Epoch 4/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6335 - loss: 0.6064 - val_accuracy: 0.7233 - val_loss: 0.5707
Epoch 5/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6983 - loss: 0.5869 - val_accuracy: 0.7200 - val_loss: 0.5708
Epoch 6/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6773 - loss: 0.6059 - val_accuracy: 0.7133 - val_loss: 0.5719
Epoch 7/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6956 - loss: 0.5806 - val_accuracy: 0.7300 - val_loss: 0.5659
Epoch 8/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6449 - loss: 0.6214 - val_accuracy: 0.7367 - val_loss: 0.5637
Ep